In [33]:
!pip install textblob
!python -m textblob.download_corpora


   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
   - ------------------------------------- 30.7/626.3 kB 259.2 kB/s eta 0:00:03
   -- ------------------------------------ 41.0/626.3 kB 326.8 kB/s eta 0:00:02
   ----- --------------------------------- 92.2/626.3 kB 521.8 kB/s eta 0:00:02
   ------ ------------------------------- 112.6/626.3 kB 595.3 kB/s eta 0:00:01
   ------- ------------------------------ 122.9/626.3 kB 450.6 kB/s eta 0:00:02
   ---------- --------------------------- 174.1/626.3 kB 551.6 kB/s eta 0:00:01
   -------------- ----------------------- 245.8/626.3 kB 654.2 kB/s eta 0:00:01
   ---------------- --------------------- 276.5/626.3 kB 654.2 kB/s eta 0:00:01
   ------------------- ------------------ 327.7/626.3 kB 724.7 kB/s eta 0:00:01
   --------------------- ---------------- 358.4/626.3 kB 717.9 kB/

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\srush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\srush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srush\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\srush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\srush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\srush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string


In [19]:
# Load data
df = pd.read_csv('blogs.csv')
df

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism
...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [20]:
# Check for missing values and basic info
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB
None


In [21]:
# Preprocessing function
def preprocess(text):
    text = text.lower()  # Convert to lowercase
    text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    tokens = text.split()  # Tokenize the text
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(tokens)


In [22]:
# Apply preprocessing to the text column
df['cleaned_text'] = df['Data'].apply(preprocess)
df['cleaned_text']

0       path cantaloupesrvcscmuedumagnesiumclubcccmued...
1       newsgroups altatheism path cantaloupesrvcscmue...
2       path cantaloupesrvcscmuedudasnewsharvardedunoc...
3       path cantaloupesrvcscmuedumagnesiumclubcccmued...
4       xref cantaloupesrvcscmuedu altatheism53485 tal...
                              ...                        
1995    xref cantaloupesrvcscmuedu talkabortion120945 ...
1996    xref cantaloupesrvcscmuedu talkreligionmisc837...
1997    xref cantaloupesrvcscmuedu talkorigins41030 ta...
1998    xref cantaloupesrvcscmuedu talkreligionmisc836...
1999    xref cantaloupesrvcscmuedu sciskeptic43561 tal...
Name: cleaned_text, Length: 2000, dtype: object

In [24]:
# Feature extraction using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['cleaned_text'])
X

<2000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 187454 stored elements in Compressed Sparse Row format>

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [26]:
# Split the data into training and test sets
y = df['Labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
# Train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)


MultinomialNB()

In [30]:
# Make predictions
y_pred = nb_model.predict(X_test)
y_pred


array(['talk.politics.misc', 'comp.sys.ibm.pc.hardware', 'sci.med',
       'rec.sport.baseball', 'sci.electronics', 'sci.electronics',
       'rec.sport.baseball', 'talk.politics.mideast', 'alt.atheism',
       'sci.med', 'alt.atheism', 'sci.med', 'sci.crypt', 'comp.windows.x',
       'comp.sys.ibm.pc.hardware', 'comp.os.ms-windows.misc', 'rec.autos',
       'comp.graphics', 'talk.politics.guns', 'talk.politics.misc',
       'comp.sys.mac.hardware', 'alt.atheism', 'alt.atheism',
       'rec.sport.hockey', 'alt.atheism', 'sci.crypt',
       'talk.politics.misc', 'rec.sport.baseball', 'rec.autos',
       'alt.atheism', 'rec.sport.baseball', 'rec.sport.hockey',
       'comp.windows.x', 'rec.sport.baseball', 'rec.sport.hockey',
       'comp.sys.mac.hardware', 'sci.med', 'sci.electronics',
       'rec.sport.hockey', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'soc.religion.christian',
       'comp.os.ms-windows.misc', 'rec.sport.baseball',
       'soc.religion.christian', 

In [31]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8225
Classification Report:
                           precision    recall  f1-score   support

             alt.atheism       0.50      0.83      0.62        18
           comp.graphics       0.79      0.83      0.81        18
 comp.os.ms-windows.misc       0.86      0.82      0.84        22
comp.sys.ibm.pc.hardware       0.76      0.76      0.76        25
   comp.sys.mac.hardware       0.83      0.90      0.86        21
          comp.windows.x       0.91      0.84      0.88        25
            misc.forsale       0.82      0.78      0.80        18
               rec.autos       0.89      0.94      0.92        18
         rec.motorcycles       0.94      0.94      0.94        16
      rec.sport.baseball       0.77      0.94      0.85        18
        rec.sport.hockey       0.88      1.00      0.94        15
               sci.crypt       0.95      0.95      0.95        19
         sci.electronics       0.62      0.62      0.62        16
                 sci.med       0.8

In [34]:
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [35]:
# Apply sentiment analysis
df['Sentiment'] = df['cleaned_text'].apply(get_sentiment)
df['Sentiment']


0       positive
1       negative
2       positive
3       positive
4       positive
          ...   
1995    positive
1996    positive
1997    positive
1998    positive
1999    positive
Name: Sentiment, Length: 2000, dtype: object

In [36]:
# Check sentiment distribution
print(df['Sentiment'].value_counts())

Sentiment
positive    1452
negative     545
neutral        3
Name: count, dtype: int64


In [37]:
# Sentiment distribution by category
sentiment_dist = df.groupby('Labels')['Sentiment'].value_counts().unstack()
print(sentiment_dist)


Sentiment                 negative  neutral  positive
Labels                                               
alt.atheism                   35.0      NaN      65.0
comp.graphics                 27.0      NaN      73.0
comp.os.ms-windows.misc       24.0      NaN      76.0
comp.sys.ibm.pc.hardware      19.0      NaN      81.0
comp.sys.mac.hardware         26.0      NaN      74.0
comp.windows.x                20.0      2.0      78.0
misc.forsale                  21.0      NaN      79.0
rec.autos                     24.0      NaN      76.0
rec.motorcycles               28.0      NaN      72.0
rec.sport.baseball            35.0      NaN      65.0
rec.sport.hockey              40.0      NaN      60.0
sci.crypt                     19.0      NaN      81.0
sci.electronics               25.0      NaN      75.0
sci.med                       34.0      NaN      66.0
sci.space                     28.0      NaN      72.0
soc.religion.christian        25.0      NaN      75.0
talk.politics.guns          

Report

1. Introduction

Brief introduction to text classification and sentiment analysis.

Explanation of the dataset and objectives.

2. Data Exploration and Preprocessing

Summary of dataset structure (number of rows, columns, and missing values).

Visualizations of category distribution and word cloud for common terms.

Detailed steps for text cleaning and feature extraction.

3. Naive Bayes Classification

Description of the Multinomial Naive Bayes model and its suitability for text data.

Training and test set size.

Performance evaluation metrics:

Accuracy: 0.82%

4. Sentiment Analysis

Summary of methodology (e.g., using TextBlob for sentiment polarity).

Key findings:

Categories with predominantly positive sentiments.

Categories with a mix of sentiments.

